In [10]:
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pandas as pd
import numpy as np

In [11]:
# Gathered data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns= boston_dataset.feature_names)

features = data.drop(["INDUS","AGE"], axis = 1)
log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=["PRICE"])
target.shape

(506, 1)

In [12]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

property_stats = features.mean().values.reshape(1, 11)

regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# Challenge: calculate the MSE and RMSE using sklearn
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [25]:
def get_log_estimate(
    nr_rooms,
    students_per_classroom,
    next_to_river= False,
    high_confidence= True,
):

    property_stats[0][RM_IDX]= nr_rooms
    log_estimate = regr.predict(property_stats)[0][0]

    if high_confidence:
        upper_bound = log_estimate+ 2*RMSE
        lower_bound = log_estimate- 2*RMSE
        interval = 95

    else:
        upper_bound = log_estimate+ RMSE
        lower_bound = log_estimate- RMSE
        interval = 68

    return log_estimate, upper_bound, lower_bound, interval

In [26]:
get_log_estimate(4,11)

(2.8273586967038216, 3.202382967098082, 2.452334426309561, 95)

In [27]:
real_median_price = 583.3
scale_factor = real_median_price/np.median(boston_dataset.target)


In [35]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """
    Estimate the price of a property in Boston
    """

    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return

    log_est, upper, lower, conf = get_log_estimate(rm, 
                                                   students_per_classroom=ptratio, 
                                                   next_to_river=chas, 
                                                   high_confidence=large_range)

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [36]:
get_dollar_estimate(rm= 2, ptratio= 10, chas=True)

The estimated property value is 388000.0.
At 95% confidence the valuation range is
USD 267000.0 at the lower end to USD 564000.0 at the high end.
